Known issues: <br>
-Lots of soundfonts don't play on FluidSynth, but works in a sfz/sf2 player. This includes: <br>
drums not working <br>
bass guitar not working (currently substituting a regular acoustic guitar for the bass sound) <br>

-Gain not adjustable, doesn't seem like midi2audio can pass gain options to FluidSynth <br>


In [1]:
from midi2audio import FluidSynth
from mido import MidiFile
import numpy as np
from scipy.io.wavfile import read, write
from IPython.display import Audio

In [2]:
#For current working directory
import os 
cwd = os.getcwd()
cwd

'C:\\Users\\Shawn\\Documents\\Georgia Tech\\MUSI 2526\\Melception'

In [ ]:
# Default filepath and soundfonts
filepath = 'MIDI\\seven_nation_army+1.mid'
soundfont = 'SoundFonts\\piano.sf2'    
fs = FluidSynth(sound_font=soundfont) #create fs object with designated soundfont

In [ ]:
# Play MIDI using selected soundfont above
FluidSynth(sound_font=soundfont).play_midi(filepath)

# Splitting MIDI

#### MIDI split function

In [3]:
def midi_to_audio(input_midi):
    try:
        # Initialize a list to hold the individual track variables
        tracks = []
        
        # Initialize program_number for use later
        program_number = None
        
        # Load MIDI file using mido.MidiFile
        mid = MidiFile(input_midi)

        # Iterate through each MIDI track and save as .mid and varaiable
        for i, track in enumerate(mid.tracks):
            # Save tracks to separate MIDI files
            output_midi = f"track_{i}.mid"
            mid_new = MidiFile(ticks_per_beat=mid.ticks_per_beat)
            mid_new.tracks.append(track)
            mid_new.save(output_midi)
            
            # Store tracks to separate variables
            globals()[f'track_{i}'] = mid_new
            
            # Print confirmation
            print(f"Track {i} saved as {output_midi} and as the variable track_{i}")
            
            
            # Check through each track for the program number            
            for msg in mid_new:
                if msg.type == 'program_change':
                    program_number = msg.program
                    break
                
            # Determine which instrument to use. If program is None then skip
            if program_number is not None:
                soundfont = timbre(program_number)
                fs = FluidSynth(sound_font=soundfont)
                
                # Output to wav file
                output_file = f"track_{i}.wav"
                fs.midi_to_audio(f'track_{i}.mid', output_file)
                # Print confirmation
                print(f"Track {i} saved as track_{i}.wav")
        
        #Combine all audio to one single wav file
        combined_samples, wav_data, sample_rates = combine_audio(mid)
        write(f'{input_midi}.wav', sample_rates[0], combined_samples)
        print(f"{input_midi} saved as {input_midi}.wav")
        
    except Exception as e:
        print(f"Error: {e}")

In [4]:
# Use program number to determine which instrument soundfont to use
def timbre(program_number):

    if program_number >=0 and program_number <=7: #Piano Timbres in MIDI are programs 0-7
        soundfont = 'SoundFonts\\piano_steinway_grand.sf2'
        
    elif program_number >=8 and program_number <=15: #Chromatic Percussion in MIDI are programs 8-15
        soundfont = 'SoundFonts\\glockenspiel.sf2' 
        
#     elif program_number >=16 and program_number <=23: #Organ Timbres in MIDI are programs 16-23
#         soundfont = 'SoundFonts\\piano.sf2' 

    elif program_number >=24 and program_number <=31: #Guitar Timbres in MIDI are programs 24-31
        soundfont = 'SoundFonts\\guitar_universal.sf2' 

    elif program_number >=32 and program_number <=39: #Bass Timbres in MIDI are programs 32-39
        soundfont = 'SoundFonts\\guitar_universal.sf2' 
        
#     elif program_number >=40 and program_number <=47: #String Timbres in MIDI are programs 40-47
#         soundfont = 'SoundFonts\\cello.sf2'

    elif program_number >=48 and program_number <=55: #Ensemble Timbres in MIDI are programs 48-55
        soundfont = 'SoundFonts\\strings_violin.sf2'

#     elif program_number >=56 and program_number <=63: #Brass Timbres in MIDI are programs 56-63
#         soundfont = 'SoundFonts\\piano.sf2'

#     elif program_number >=64 and program_number <=71: #Reed Timbres in MIDI are programs 64-71
#         soundfont = 'SoundFonts\\piano.sf2'

    elif program_number >=72 and program_number <=79: #Pipe Timbres in MIDI are programs 72-79
        soundfont = 'SoundFonts\\flute.sf2'

#     elif program_number >=80 and program_number <=87: #Synth Lead in MIDI are programs 80-87
#         soundfont = 'SoundFonts\\piano.sf2'

#     elif program_number >=88 and program_number <=95: #Synth Pad in MIDI are programs 88-95
#         soundfont = 'SoundFonts\\piano.sf2'

    else:
        print(f"Error: Timbre not yet supported")
        
    return soundfont

In [5]:
def normalize(samples):
    # Find the maximum absolute value in the samples
    max_val = np.max(np.abs(samples))
    # Scale samples to the range of int16
    if max_val > 0:
        samples = samples / max_val * 32767
    return samples.astype(np.int16)

In [6]:
def pad_audio(samples, target_length):
    if len(samples) < target_length:
        pad_length = target_length - len(samples)
        samples = np.pad(samples, (0, pad_length), 'constant')
    return samples

In [7]:
def combine_audio(mid):
    
    wav_data = []
    sample_rates = []
    max_length = 0
    
    # Iterate through all available tracks
    for i, track in enumerate(mid.tracks[1:], start=1):
        fs, x = read(f"track_{i}.wav") #Read wav file
        x = x[:,0] #Make audio mono
        wav_data.append(x) #Add to wav_data array of all audio samples
        sample_rates.append(fs) #Add to sample_rates array of all sample rates
        if len(x) > max_length:
            max_length = len(x)
    
    # Pad all WAV files to the maximum length
    for i in range(len(wav_data)):
        wav_data[i] = pad_audio(wav_data[i], max_length)
    
    # Combine the WAV files
    combined_samples = np.sum(wav_data, axis=0)
    combined_samples = normalize(combined_samples)
    
    # Write the combined samples to the output file
    write('output.wav', sample_rates[0], combined_samples)
    
    return combined_samples, wav_data, sample_rates

In [8]:
#Testing
input_midi = 'MIDI\\honestly_result_flute.mid'  # Replace with your input MIDI file
midi_to_audio(input_midi)

Track 0 saved as track_0.mid and as the variable track_0
Track 1 saved as track_1.mid and as the variable track_1
Track 1 saved as track_1.wav
Track 2 saved as track_2.mid and as the variable track_2
Track 2 saved as track_2.wav
Track 3 saved as track_3.mid and as the variable track_3
Track 3 saved as track_3.wav
Track 4 saved as track_4.mid and as the variable track_4
Track 4 saved as track_4.wav
Track 5 saved as track_5.mid and as the variable track_5
Track 5 saved as track_5.wav
Track 6 saved as track_6.mid and as the variable track_6
Track 6 saved as track_6.wav
MIDI\honestly_result_flute.mid saved as MIDI\honestly_result_flute.mid.wav
